In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Generics: Array-based stack

<div class="alert alert-block alert-success"> 
    Source code for this notebook can be found in the package <tt>ca.queensu.cisc124.notes.generics.basics</tt>.
</div>

Readers should review the array-based stack of strings implementation described in the [Interfaces: Array-based stack notebook](../part3/interfaces_array_based_stack.ipynb#notebook_id).

The implementation of the array-based stack of strings implementation is shown in the next cell. The lines of code that we need to modify to create a generic implementation
are indicated with a numbered comment.

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Arrays;
import ca.queensu.cs.cisc124.notes.interfaces.Stack;

public class ArrayStack implements Stack {                                     // 1, 2

    // the initial capacity of the stack
    private static final int DEFAULT_CAPACITY = 16;

    // the array that stores the stack
    private String[] arr;                                                      // 3

    // the index of the top of the stack (equal to -1 for an empty stack)
    private int top;

    public ArrayStack() {
        this.arr = new String[ArrayStack.DEFAULT_CAPACITY];                    // 4
        this.top = -1;
    }
    
    @Override
    public int size() {
        return this.top + 1;
    }

    @Override
    public void push(String elem) {                                            // 5
        // do we need to resize the array?
        if (this.size() == this.arr.length) {
            this.arr = Arrays.copyOf(this.arr, this.arr.length * 2);
        }
        this.top++;
        this.arr[this.top] = elem;
    }

    @Override
    public String pop() {                                                      // 6
        // is the stack empty?
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        // get the element at the top of the stack
        String element = this.arr[this.top];                                   // 7, 8
        
        // null out the value stored in the array
        this.arr[this.top] = null;

        // adjust the top of stack index
        this.top--;

        // return the element that was on the top of the stack
        return element;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.arr[i]);
            }
        }
        return b.toString();
    }
    
}

## Implementing a generic array-based stack

Only a small number of changes are required to convert our existing stack of strings implementation to a generic implementation.

### Declaring the class

The class declaration (commented line `// 1, 2`) is modified to include a generic type variable `E` for the element type; similarly, we also need to indicate that we are implementing
the generic interface. The modified line becomes:

```java
public class ArrayStack<E> implements Stack<E> {
```


### Fields

There is only one slightly complicating factor when implementing a generic array-based stack: Generics as implemented in Java do not mix well with arrays.
In particular, Java does not allow the programmer to create an array of a generic variable type:

**Exercise 1** Run the following cell that contains a method that tries to create an array of generic type. Does the class compile?

In [ ]:
public class GenericArray {
    
    public static <E> E[] makeArray(int size) {
        // try to create an array using the generic type variable
        return new E[size]; 
    }
}

The reason for not allowing arrays of generic variable type is related to the way that generics were added to the Java language.

<div class="alert alert-block alert-info">
    Java generics were implemented using <i>type erasure</i>. In brief, type erasure means that the compiler removes all information about generic type variables from the
    compiled byte code. At runtime, there is no mechanism available to determine the actual types that were used for the generic type variables (for example,
    there is no way to determine what the element type is in a <tt>List</tt> object at runtime). Java arrays are different in that the element type must always be
    available at runtime, which is impossible if generic arrays were allowed.<br />
    Interested readers can learn more by referring to the official <a href="https://docs.oracle.com/javase/tutorial/java/generics/erasure.html">Type erasure tutorial</a>.
</div>

Instead of using an array of the generic type variable, we will use an array of `Object`.
As we saw in the previous notebook, an array of type `Object[]` can store any element type. We do not lose type safety by using such an array because the
`push` method accepts only elements of the correct type and the `pop` method returns references of the correct type.

The commented line `// 3` becomes:

```java
private Object[] arr;
```


### Constructor

The constructor now creates an array of `Object` instead of an array of `String`. The commented line `// 4` becomes:

```java
this.arr = new Object[ArrayStack.DEFAULT_CAPACITY];
```

### `push` method

The `push` method now accepts a parameter of the generic type variable `E`.  The commented line `// 5` becomes:

```java
public void push(E elem) {
```

### `pop` method

Three minor changes are required in the `pop` method. First, `pop` now returns a reference to the generic type variable. The commented line `// 6` becomes:

```java
public E pop() {
```

Second, the type of the popped element is the same as the generic type variable `E`. The beginning of the commented line `// 7, 8` becomes:


```java
E elem =
```

Third, the stack elements are stored in an array of `Object` but we know that the actual type is the same as the generic type variable `E`. We downcast the array element to type `E` so that
the commented line `// 7, 8` becomes:

```java
E elem = (E) this.arr[this.top];
```

That's it! We now have a fully functioning array-based generic stack. The class is shown in the following cell:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Arrays;
import ca.queensu.cs.cisc124.notes.generics.basics.Stack;

public class ArrayStack<E> implements Stack<E> {                                     // 1, 2

    // the initial capacity of the stack
    private static final int DEFAULT_CAPACITY = 16;

    // the array that stores the stack
    private Object[] arr;                                                            // 3

    // the index of the top of the stack (equal to -1 for an empty stack)
    private int top;

    public ArrayStack() {
        this.arr = new Object[ArrayStack.DEFAULT_CAPACITY];                          // 4
        this.top = -1;
    }
    
    @Override
    public int size() {
        return this.top + 1;
    }

    @Override
    public void push(E elem) {                                                       // 5
        // do we need to resize the array?
        if (this.size() == this.arr.length) {
            this.arr = Arrays.copyOf(this.arr, this.arr.length * 2);
        }
        this.top++;
        this.arr[this.top] = elem;
    }

    @Override
    public E pop() {                                                                 // 6
        // is the stack empty?
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        // get the element at the top of the stack as type E
        E element = (E) this.arr[this.top];                                          // 7, 8
        
        // null out the value stored in the array
        this.arr[this.top] = null;

        // adjust the top of stack index
        this.top--;

        // return the element that was on the top of the stack
        return element;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.arr[i]);
            }
        }
        return b.toString();
    }
    
}


We can run the same test program found in the [Generic classes and interfaces notebook](./generics_list_based_stack_implementation.ipynb#test_program) by changing `ListStack` to `ArrayStack`:

In [ ]:
%classpath add jar ../resources/jar/notes.jar

// make sure to run the previous code cell to compile the Stack interface and ListStack class first
import ca.queensu.cs.cisc124.notes.generics.basics.Stack;

Stack<String> s = new ArrayStack<>();
s.push("a");
s.push("b");
s.push("c");
System.out.println(s);
System.out.println();

Stack<Integer> t = new ArrayStack<>();
t.push(1);
t.push(2);
t.push(3);
System.out.println(t);
System.out.println();

Stack<Double> u = new ArrayStack<>();
u.push(1.0);
u.push(2.0);
u.push(3.0);
System.out.println(u);
System.out.println();

## Exercises

2. Suppose that we decided that the top element of the stack should always be located at index `0` of the array.
    1. What would the `push` method have to do every time an element was pushed on to the stack?
    2. What would the `pop` method have to do every time an element was popped off the stack?
    
3. An alternative implementation sets `this.top` to `this.arr.length` for an empty stack. Pushing an element on to the stack causes `this.top` to move towards the front of the array. Popping an element from the stack causes `this.top` to move towards the back of the array. Are there any advantages or disadvantages of this implementation compared to the implementation described in the notebook?

4. See Exercises 1-9 of the [previous notebook](./generics_list_based_stack.ipynb#notebook_id). Repeat the exercises for an array-based stack (Exercises 1-3 are already done for you below). For the static
factory method (Exercise 9) the input stack should be an `ArrayStack` object and the returned stack should be an `ArrayStack` object.

In [ ]:
// Exercise 4 (Parts 1-3; already done for you)
public interface Stack<E> {

    public int size();

    default boolean isEmpty() {
        return this.size() == 0;
    }

    public void push(E elem);

    public E pop();

    public E peek();

    public boolean contains(E elem);

    public void reverse();
}

In [ ]:
// Exercise 3 (Parts 4-9)
import java.util.Arrays;
import java.util.Collection;

public class ArrayStack<E> implements Stack<E> {

    // the initial capacity of the stack
    private static final int DEFAULT_CAPACITY = 16;

    // the array that stores the stack
    private Object[] arr;

    // the index of the top of the stack (equal to -1 for an empty stack)
    private int top;

    public ArrayStack() {
        this.arr = new Object[ArrayStack.DEFAULT_CAPACITY];
        this.top = -1;
    }
    
    @Override
    public int size() {
        return this.top + 1;
    }

    @Override
    public void push(E elem) {
        // do we need to resize the array?
        if (this.size() == this.arr.length) {
            this.arr = Arrays.copyOf(this.arr, this.arr.length * 2);
        }
        this.top++;
        this.arr[this.top] = elem;
    }

    @Override
    public E pop() {
        // is the stack empty?
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        // get the element at the top of the stack as type E
        E element = (E) this.arr[this.top];
        
        // null out the value stored in the array
        this.arr[this.top] = null;

        // adjust the top of stack index
        this.top--;

        // return the element that was on the top of the stack
        return element;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.arr[i]);
            }
        }
        return b.toString();
    }
}